In [1]:
%load_ext sql

In [2]:
%sql sqlite:///../../20-lab-sql-select/publications.db

'Connected: @../../20-lab-sql-select/publications.db'

## Challenge 1 - Most Profiting Authors

### Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

* Title ID
* Author ID
* Royalty of each sale for each author

In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

In [3]:
%%sql
SELECT tit.title_id AS title_id,
       titaut.au_id AS author_id,
       round(tit.price * sal.qty * tit.royalty / 100 * titaut.royaltyper / 100, 2) AS individual_royalty
FROM titles tit
     INNER JOIN titleauthor titaut ON titaut.title_id = tit.title_id
     INNER JOIN sales sal ON tit.title_id = sal.title_id
LIMIT 5

 * sqlite:///../../20-lab-sql-select/publications.db
Done.


title_id,author_id,individual_royalty
PS3333,172-32-1176,29.99
BU1032,213-46-8915,4.0
BU1032,213-46-8915,8.0
BU2075,213-46-8915,25.12
PC1035,238-95-7766,110.16


### Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

* Title ID
* Author ID
* Aggregated royalties of each title for each author
    * Hint: use the *SUM* subquery and group by both `au_id` and `title_id`

In the output of this step, each title should appear only once for each author.

In [4]:
%%sql
SELECT tit.title_id AS title_id,
       titaut.au_id AS author_id,
       round(sum(tit.price * sal.qty * tit.royalty / 100 * titaut.royaltyper / 100), 2) AS total_royalty
FROM titles tit
     INNER JOIN titleauthor titaut ON titaut.title_id = tit.title_id
     INNER JOIN sales sal ON tit.title_id = sal.title_id
GROUP BY titaut.au_id, tit.title_id
LIMIT 5

 * sqlite:///../../20-lab-sql-select/publications.db
Done.


title_id,author_id,total_royalty
PS3333,172-32-1176,29.99
BU1032,213-46-8915,11.99
BU2075,213-46-8915,25.12
PC1035,238-95-7766,110.16
BU1111,267-41-2394,11.95


### Step 3: Calculate the total profits of each author

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

* Author ID
* Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.

In [5]:
%%sql
SELECT author_id,
       (total_royalty + advance) AS profit
FROM (SELECT titaut.au_id AS author_id,
       round(sum(tit.price * sal.qty * tit.royalty / 100 * titaut.royaltyper / 100), 2) AS total_royalty,
       tit.advance as advance
FROM titles tit
     INNER JOIN titleauthor titaut ON titaut.title_id = tit.title_id
     INNER JOIN sales sal ON tit.title_id = sal.title_id
GROUP BY titaut.au_id, tit.title_id)
ORDER BY profit desc
LIMIT 3

 * sqlite:///../../20-lab-sql-select/publications.db
Done.


author_id,profit
722-51-5454,15021.53
899-46-2035,15007.18
213-46-8915,10150.12


## Challenge 2

Elevating from your solution in Challenge 1 , create a table named `most_profiting_authors` to hold the data about the most profiting authors. The table should have 2 columns:

* `au_id` - Author ID
* `profits` - The profits of the author aggregating the advances and royalties

In [6]:
%%sql
CREATE TABLE most_profiting_authors AS
SELECT author_id,
       (total_royalty + advance) AS profit
FROM (SELECT titaut.au_id AS author_id,
       round(sum(tit.price * sal.qty * tit.royalty / 100 * titaut.royaltyper / 100), 2) AS total_royalty,
       tit.advance as advance
FROM titles tit
     INNER JOIN titleauthor titaut ON titaut.title_id = tit.title_id
     INNER JOIN sales sal ON tit.title_id = sal.title_id
GROUP BY titaut.au_id, tit.title_id)
ORDER BY profit desc

 * sqlite:///../../20-lab-sql-select/publications.db
Done.


[]

In [7]:
%%sql
SELECT *
FROM most_profiting_authors
LIMIT 5

 * sqlite:///../../20-lab-sql-select/publications.db
Done.


author_id,profit
722-51-5454,15021.53
899-46-2035,15007.18
213-46-8915,10150.12
427-17-2319,8050.0
846-92-7186,8050.0
